## 2 кейс

**Соберите данные из Google Sheets и напишите функцию `generate_report`, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет отчет в файл `student_debt_report.txt`**

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [ ]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

--2025-11-28 11:29:56--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 140.82.121.4
Connecting to gist.github.com (gist.github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2025-11-28 11:29:57--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: ‘creds.json’

creds.json          100%[===================>]   2.30K  --.-KB/s    in 0s      

2025-11-28 11:29:57 (38.6 MB/s) - ‘creds.json’ saved [2

Чтобы посмотреть как выглядят сами гугл таблицы в оригинале - можете перейти по [ссылке](https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing) и изучить.

Небольшое описание столбцов в таблицах:  
**Лист1:** `student_id` - айди студента; `student_name` - имя студента; `installment` (Y/N) - факт наличия рассрочки у студента (Y - рассрочка есть, N - рассрочки нет).  
**Лист2:** `student_id` - айди студента; `last_payment_date` - дата последней полученной оплаты; `expected_payment_date` - дата, в которую ожидаем следующий платеж (рассчитывается от даты последнего платежа).  
**Лист3:** `student_id` - айди студента; `already_payed_amount` - уже выплаченная часть рассрочки; `left_to_pay` - оставшаяся (невыплаченная) часть; `one-time_payment` - единоразовый платеж; `installment_amount` - общая сумма, которая бралась в рассрочку.

Как примерно должен выглядеть результат:

In [ ]:
with open('result.txt') as f:
  print(f.read())

Студент Иванов У.У. - долг 100000 рублей
Студент Петров Е.Е. - долг 250000 рублей
Студент Сидоров И.И. - долг 10000 рублей


In [ ]:
#@title Если возникнут трудности с определением `scope`, подсказка:

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

**Примечание**

Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на **1 марта 2023 года**. А периоды внесения платежей для всех студентов одинаковы - **6 месяцев** (183 дня).

То есть, если ожидаемый платеж должен был пройти 3 февраля 2022 года, то к 1 марта 2023 студент просрочил уже три платежа (3 февраля 2022, 3 августа 2022 и 3 февраля 2023). При расчетах отталкивайтесь от даты ожидаемого платежа, разницу между платежами берите 183 дня.

**Решение**

Напишите свое решение ниже

In [ ]:
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date

# Указываем необходимые права доступа к таблицам
scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

# Загружаем ключи аутентификации из файла json
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)

# Авторизуемся в Google Sheets API
client = gspread.authorize(creds)

In [ ]:
# Открываем таблицу по ID
spreadsheet_id = '1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U'
f = client.open_by_key(spreadsheet_id)
# Переменные для работы с листами
sheet_01 = f.get_worksheet(0)
sheet_02 = f.get_worksheet(1)
sheet_03 = f.get_worksheet(2)

sheet1 = sheet_01.get_all_values() #список по первому листу
sheet2 = sheet_02.get_all_values() #список по второму листу
sheet3 = sheet_03.get_all_values() #список по третьему листу

In [ ]:
def generate_report(sheet1, sheet2, sheet3):

# 1. Переводим листы в дф и объединяем их в один по student_id
    df1 = pd.DataFrame(sheet1)
    df2 = pd.DataFrame(sheet2)
    df3 = pd.DataFrame(sheet3)
    df_combined = df1.merge(df2, on='student_id').merge(df3, on='student_id')

# 2. Измененение типов данных
    df_combined['expected_payment_date'] = pd.to_datetime(df_combined['expected_payment_date'], format='%d.%m.%Y', errors='coerce')
    df_combined['last_payment_date'] = pd.to_datetime(df_combined['last_payment_date'])
    df_combined['one-time_payment'] = pd.to_numeric(df_combined['one-time_payment'], errors='coerce')

# 3. Фильтруем датафрейм по дате целевой дате (01.03.2023)
    target_date = pd.to_datetime('01.03.2023', format='%d.%m.%Y') # задаем целевую дату в формате datetime
    df_combined = df_combined[df_combined['expected_payment_date'] <= target_date] # фильтруем дф по критерию дата платежа не ранее таргета

# 4. Расчет долга
    days_diff = (target_date - df_combined['expected_payment_date']).dt.days # разница в днях
    periods = np.ceil(days_diff / 183) # количество пропущенных платежей (раз в 183 дня), окргляем вверх
    df_combined['debt'] = (periods * df_combined['one-time_payment']).astype('Int64') # расчетная сумма долга

### Поправка: расчетная сумма долга не может превышать сумму оставшегося долга
    df_combined['debt'] = np.minimum(df_combined['debt'], df_combined['left_to_pay'])

# 5. Формируем список должников в txt
    with open("./student_debt_report.txt", "w", encoding="utf-8") as f: # создали файл для запииси и задали алиас
        lines = [f"Студент {row['student_name']} - долг {row['debt']} рублей" for _, row in df_combined.iterrows()]
        # используем генератор списков [for _, row ...] и берем нужные значения из
        f.write("\n".join(lines))


In [ ]:
generate_report(sheet1, sheet2, sheet3)

/tmp/ipython-input-705825183.py:11: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_combined['last_payment_date'] = pd.to_datetime(df_combined['last_payment_date'])


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

import pandas as pd

user_answer = pd.read_csv('student_debt_report.txt')
correct_answer = pd.read_csv('student_debt.txt')

--2026-01-10 21:31:49--  https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt [following]
--2026-01-10 21:31:49--  https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11325 (11K) [text/plain]
Saving to: ‘student_debt.txt’

student_debt.txt    100%[===================>]  11.06K  --.-KB/s    in 0s      

2026-01-10 21:31:49 (58.7 MB/s)

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
